In [1]:
# ==============================================================================
# 📦 0. INSTALACE A NAČTENÍ KNIHOVEN (START)
# ==============================================================================

# Seznam potřebných balíčků
required_packages <- c("readxl", "dplyr", "openxlsx", "rstatix", "tidyverse", "moments", "vcd")

# Funkce: Nainstaluj jen to, co chybí, a pak vše načti
install_and_load <- function(pkg) {
  if (!require(pkg, character.only = TRUE)) {
    install.packages(pkg, dependencies = TRUE)
    library(pkg, character.only = TRUE)
  }
}

# Spuštění pro všechny balíčky najednou
lapply(required_packages, install_and_load)

# Pro jistotu explicitní načtení (abys viděl, že to funguje):
library(readxl)    # Čtení Excelů
library(openxlsx)  # Zápis Excelů
library(tidyverse) # Obsahuje dplyr, ggplot2, tidyr (na filtrace a grafy)
library(rstatix)   # Statistické testy (moderní verze)
library(moments)   # Šikmost a špičatost (skewness, kurtosis)
library(vcd)       # Kategoriální data (Cramerovo V, Mozaikové grafy)

print("✅ Všechny knihovny jsou úspěšně načteny!")

Loading required package: readxl

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: openxlsx

Loading required package: rstatix


Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter


Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.1     ✔ readr     2.1.6
✔ ggplot2   4.0.0     ✔ stringr   1.5.2
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ purrr     1.1.0     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ rstatix::filter() masks dplyr::filter(), stats::filter()
✖ dplyr::lag()      masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loa

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[1] "✅ Všechny knihovny jsou úspěšně načteny!"


In [2]:
data <- readxl::read_excel("dataset_1_out.xlsx", skip = 0)
head(data)

id,vyrobce,propustnost_puvodni,propustnost_nova,montaz
<dbl>,<chr>,<dbl>,<dbl>,<chr>
1,A,285.36,295.36,amater
2,A,295.83,308.15,profesional
3,A,286.07,299.70,amater
4,A,289.01,298.55,amater
5,A,292.73,304.04,amater
6,A,290.00,304.09,profesional


In [4]:
x <- data %>% 
  filter(vyrobce == "C") %>%  # <--- ZKONTROLUJ PŘESNÝ NÁZEV KATEGORIE!
  pull(propustnost_puvodni)      # pull() vytáhne sloupec jako vektor čísel

# 2. VÝPOČET HRADEN (MEZÍ)
# ------------------------------------------------
# na.rm = TRUE je pojistka proti prázdným buňkám
Q1 <- quantile(x, 0.25, na.rm = TRUE)
Q3 <- quantile(x, 0.75, na.rm = TRUE)
IQR_hodnota <- IQR(x, na.rm = TRUE)    # Rozdíl Q3 - Q1

# Výpočet vnitřních hradeb (1.5 násobek IQR)
DM <- Q1 - 1.5 * IQR_hodnota # Dolní mez (Lower Fence)
HM <- Q3 + 1.5 * IQR_hodnota # Horní mez (Upper Fence)

print("--- HRANICE (Vnitřní hradby) ---")
print(paste("Dolní mez (DM):", round(DM, 4)))
print(paste("Horní mez (HM):", round(HM, 4)))
print(paste("Interval normálních hodnot: <", round(DM, 2), ";", round(HM, 2), ">"))

# 3. IDENTIFIKACE ODLEHLÝCH POZOROVÁNÍ
# ------------------------------------------------
# Hodnoty, které jsou MENŠÍ než DM nebo VĚTŠÍ než HM
odlehle <- x[x < DM | x > HM]

print("--- ODLEHLÉ HODNOTY (Outliers) ---")
if (length(odlehle) > 0) {
  print(odlehle)
  print(paste("Počet odlehlých:", length(odlehle)))
} else {
  print("Žádná odlehlá pozorování nebyla nalezena.")
}

# 4. VÝPOČET STATISTIK PO OČIŠTĚNÍ (BEZ ODLEHLÝCH)
# ------------------------------------------------
# Vybereme jen to, co je UVNITŘ hradeb
x_clean <- x[x >= DM & x <= HM]

print("--- STATISTIKY PO ODSTRANĚNÍ ODLEHLÝCH ---")
print(paste("Nový průměr:", round(mean(x_clean, na.rm = TRUE), 4)))
print(paste("Nový medián:", round(median(x_clean, na.rm = TRUE), 4)))
print(paste("Nový horní kvartil (Q3):", round(quantile(x_clean, 0.25, na.rm = TRUE), 4)))
print(paste("Nová směrodatná odchylka:", round(sd(x_clean, na.rm = TRUE), 4)))

[1] "--- HRANICE (Vnitřní hradby) ---"
[1] "Dolní mez (DM): 273.1312"
[1] "Horní mez (HM): 306.6613"
[1] "Interval normálních hodnot: < 273.13 ; 306.66 >"
[1] "--- ODLEHLÉ HODNOTY (Outliers) ---"
[1] 270.55 312.21 312.21
[1] "Počet odlehlých: 3"
[1] "--- STATISTIKY PO ODSTRANĚNÍ ODLEHLÝCH ---"
[1] "Nový průměr: 289.6369"
[1] "Nový medián: 289.69"
[1] "Nový horní kvartil (Q3): 285.78"
[1] "Nová směrodatná odchylka: 5.9018"
